# **Curso IA desde Cero**

* Dr. Irvin Hussein López Nava
* M.C. Joan M. Raygoza Romero

## Librerias

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from skimage.feature import hog
import os
import pandas as pd

## Funciones

### Hog
Descriptor que representa una imagen mediante los patrones de bordes y contornos.

En lugar de usar los valores de los píxeles, HOG analiza cómo cambian las intensidades para obtener orientaciones de gradiente en pequeñas regiones (celdas).

El proceso es:

1. Calcular gradientes en cada píxel (dirección y magnitud del cambio de intensidad).
2. Agrupar los gradientes en celdas.
3. Construir histogramas de orientaciones en cada celda.
4. Normalizar por bloques para que sea robusto a iluminación.
5. Concatenar todos los histogramas → vector final de características.

In [ ]:
def hog_features(img):
    #img: imagen en escala de grises (2D)
    features = hog(
        img,
        orientations=6,
        pixels_per_cell=(7, 7),
        cells_per_block=(4, 4),
        block_norm='L2-Hys'
    )
    return features

### Momentos Hu
Son un conjunto de 7 valores numéricos que describen la forma global de un objeto en una imagen.

Son como resúmenes matemáticos de la forma.

Se calculan a partir de los momentos geométricos de la imagen y tienen una propiedad muy importante.

Son invariantes a:

* traslación (si se mueve el objeto)
* rotación (si gira)
* escala (si cambia de tamaño)

In [ ]:
def hu_moments(img):
    #img: imagen en escala de grises (2D)
    moments = cv2.moments(img)
    hu = cv2.HuMoments(moments).flatten()  # 7 momentos

    # Suele aplicarse log para estabilizar los valores
    hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-7)

    return hu_log  # tamaño = 7

### Extracción de características

In [ ]:
def extract_features(img):
    f_hog = hog_features(img)                   # 96
    f_hu = hu_moments(img)                      # 7

    # Vector final de características
    features = np.concatenate([f_hog, f_hu])
    return features  # tamaño total = 96 + 7 = 103


## Path de donde estan mis imagenes

In [ ]:
base_path = 'drive/MyDrive/IA desde cero/datasets/mnist_digits/'

### Extracción de características por cada imagen

In [ ]:
features = []
labels = []
for class_name in os.listdir(base_path):
    print(class_name)
    for image_name in os.listdir(base_path + class_name):
        filename = base_path + class_name + '/' + image_name
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        features.append(extract_features(img))
        labels.append(class_name)

9
4
6
2
8
3
7
1
0
5


### Creación del conjunto de datos

In [ ]:
features_df = pd.DataFrame(features, columns=['hog_'+str(i) for i in range(96)]+['hu_'+str(i) for i in range(7)])
features_df['label'] = labels
features_df.head()

,hog_0,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,hog_8,hog_9,...,hog_94,hog_95,hu_0,hu_1,hu_2,hu_3,hu_4,hu_5,hu_6,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.030722,0.104261,0.063024,0.048946,...,0.002733,0.00511,2.607229,5.994159,6.981098,6.998622,7.0,6.999999,-7.0,9
1,0.0,0.0,0.0,0.0,0.0,0.0,0.025109,0.012114,0.036678,0.020856,...,0.000285,0.00000,2.683464,5.734548,6.992246,6.998122,7.0,6.999998,7.0,9
2,0.0,0.0,0.0,0.0,0.0,0.0,0.021115,0.100505,0.069573,0.013670,...,0.000000,0.00000,2.699988,5.738301,6.989513,6.998037,7.0,6.999998,-7.0,9
3,0.0,0.0,0.0,0.0,0.0,0.0,0.023599,0.007399,0.006840,0.006550,...,0.000000,0.00000,2.686197,5.740604,6.990049,6.999376,-7.0,-6.999999,7.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,2.859548,6.367667,6.998003,6.999938,-7.0,7.000000,7.0,9


### Guardar el conjunto de datos

In [ ]:
features_df.to_csv('mnist_digits_features.csv', index=False)